In [ ]:
PLAYER_NAME = "Dave"                    # Player of interest
MOVES_SHORT = 25                        # Short game threshold
ACPL_MIN_SHORT = 25                     # Floor for ACPL in short games
ACPL_MIN_NORMAL = 20                    # Floor for ACPL in normal-length games
R_MIN = 800                             # Floor for the worst performances
R_MAX = 2300                            # Ceiling for the best performances
K = 0.05341958625524604                 # Model decay constant
ACPL_BEST = 23.709467837259233          # Model best ACPL
WINDOW = 7                              # Rolling window size (days) for rolling average performance score

In [ ]:
%run constants.ipynb
%run database.ipynb
%run analysis.ipynb
%run pathutils.ipynb
%run export.ipynb

In [ ]:
import numpy as np

def calculate_performance(acpl, ACPL_best=ACPL_BEST):
    return R_MIN + (R_MAX - R_MIN) * np.exp(-K * (acpl - ACPL_best))

In [ ]:
import pandas as pd

# Generate the ACPL data set
connection = connect()
player_acpl_df = generate_acpl_for_player(connection, PLAYER_NAME, [""], 1, MOVES_SHORT, ACPL_MIN_SHORT, ACPL_MIN_NORMAL)

# Get a distinct list of game IDs
game_ids = player_acpl_df["game_id"].unique()
for game_id in game_ids:
    # Load the game metadata for the game and extract the date
    metadata_df = load_game_metadata(connection, game_id)
    date_row = metadata_df.loc[metadata_df["name"] == "Date", "value"]
    date = date_row.iloc[0]

    # Apply the date to the ACPL data frame
    player_acpl_df.loc[player_acpl_df["game_id"] == game_id, "date" ] = date

# Remove all rows that don't have a date
player_acpl_df = player_acpl_df[player_acpl_df["date"].notna()]

# Calculate the average ACPL by date
average_acpl_df = (
    player_acpl_df
    .groupby("date", as_index=False)["acpl"]
    .mean()
).sort_values("date")

# Calculate the score, using the model
average_acpl_df["performance"] = calculate_performance(average_acpl_df["acpl"])

# Make the date the index
average_acpl_df["date"] = pd.to_datetime(average_acpl_df["date"], errors="raise")
average_acpl_df.set_index("date", inplace=True)

# Preview the data
display(average_acpl_df)

In [ ]:
# Calculate the monthly average performance
monthly_perf_df = (
    average_acpl_df
    .resample("M")
    .agg(
        avg_performance=("performance", "mean"),
        games=("performance", "count")
    )
    .dropna()
    .reset_index()
)

# Preview the data
display(monthly_perf_df)

In [ ]:
import pandas as pd

export_to_spreadsheet(f"personal-performance-rating-{PLAYER_NAME}", {
    "Performance Data": average_acpl_df,
    "Monthly Average": monthly_perf_df
})

In [ ]:
import matplotlib.pyplot as plt

monthly_perf_df["month"] = monthly_perf_df["date"].dt.strftime("%Y-%m")

plt.figure(figsize=(12, 5))
plt.bar(
    monthly_perf_df["month"],
    monthly_perf_df["avg_performance"]
)

plt.xlabel("Month")
plt.ylabel("Average Performance")
plt.title("Monthly Average Performance")
plt.tight_layout()

# Export the chart
export_chart(f"personal-performance-rating-{PLAYER_NAME}", "", "png")

plt.show()